In [ ]:
import numpy as np
import glob
from astropy.coordinates import SkyCoord
from astropy.coordinates import ICRS, Galactic, FK4, FK5
from astropy.coordinates import Angle, Latitude, Longitude
import astropy.units as un
from astropy.time import Time
import pandas as pd

# Match flare type stars with LSPs

In this code we check whether any flare type objects are in the field of view for MeerKAT LSPs.

In the final data frame, there is a row for every flare star that is within the primary beam of an LSP. That means that some flare stars are counted twice, if an LSP has overlapping beams and the flare star falls in both. You can easily remove stars that are counted twice by using drop_duplicates, I'll demonstrate this near the bottom of the notebook.

In [ ]:
# Read in the flare objects file
flarestars = pd.read_csv('FlareTypeStars_Pandas_SimbadNames.csv').drop(['Unnamed: 0'],
                                                                       axis=1)
# Get the coordinates of each source
fstar_coords = SkyCoord(flarestars['ra(deg)'].astype(float)*un.degree,
                        flarestars['dec(deg)'].astype(float)*un.degree,
                        pm_ra_cosdec=(np.array(flarestars['pmra(arcsec/yr)']).astype(float)*un.mas/un.yr),
                        pm_dec=(np.array(flarestars['pmdec(arcsec/yr)']).astype(float)*un.mas/un.yr),
                        obstime=Time('2019-01-01T00:00:00.00'))

# Read in the LSP catalogue
lsps = pd.read_csv('MeerKAT_LSP_AllPointings.csv')
# Remove MeerTIME, since these coordinates
# are educated guesses, and the integration
# time per pointing is very short
lsps_nomeertime = lsps[lsps['LSP'] != 'pulsars']
# Get the LSP coordinates
lsp_coords = SkyCoord(lsps_nomeertime['RA(deg)'].astype(float)*un.degree,
                      lsps_nomeertime['DEC(deg)'].astype(float)*un.degree,
                      obstime=Time('2019-01-01T00:00:00.00'))

In [ ]:
# Set up the columns for your final
# data frame by combining the flare star
# (FSTAR) and LSP (LSP) file
# columns
cols = []
for col in flarestars.columns:
    cols.append('FSTAR '+col)
for col in lsps.columns:
    if '#' in col:
        cols.append('LSP index')
    elif 'LSP' in col:
        cols.append('LSP')
    else:
        cols.append('LSP '+col.strip())
cols.append('min_sep(deg)')

matched_sources = []
# For each LSP pointing, check which
# stars are within the primary beam
# at the centre of the observing band
for index, row in lsps_nomeertime.iterrows():
    lsp_coord = SkyCoord(row['RA(deg)']*un.degree,
                         row['DEC(deg)']*un.degree,
                         obstime=Time('2019-01-01T00:00:00.00'))
    beam_size = float(row['Beam_FWHM_Diameter(deg)'])
    seps = lsp_coord.separation(fstar_coords)
    in_beam = np.where(seps.deg<0.5*beam_size)[0]

    if len(in_beam)> 0:
        for b in in_beam:
            source_info = []
            for col in flarestars.columns:
                fstar_b = flarestars.iloc[b]
                source_info.append(str(fstar_b[col]))
            for col in lsps.columns:
                source_info.append(str(row[col]))
            source_info.append(str(seps.deg[b]))
            matched_sources.append(source_info)
matched_sources = np.array(matched_sources)

# Take your matched sources and put
# them in a data frame
new_table = dict()
for c, col in enumerate(cols):
    new_table[col] = matched_sources[:, c]
fstar_lsps = pd.DataFrame(data=new_table)

# Save that data frame
fstar_lsps.to_csv('FStar_LSP_Matches.csv', index=False)

fstar_lsps is the final data frame with all of the matches. But maybe you want just the unique stars that are in at least one LSP. Then you can use drop_duplicates to get rid of copies of the same star.

In [ ]:
unique_fstars = fstar_lsps.drop_duplicates(subset=['FSTAR ra(deg)', 'FSTAR dec(deg)'])

Now you can, for example, have a look at just the SIMBAD RS CVns.

In [ ]:
unique_fstars[unique_fstars['FSTAR Catalogue']=='SimbadRSCVns']

Or you could have a look at all the matches for a single LSP, such as THUNDERKAT

In [ ]:
unique_fstars[unique_fstars['LSP']=='ThunderKAT']